# Fact Orders

In [0]:
df = spark.sql("select * from retail_databricks_catalog.silver_schema.orders_silver")
df.display()

In [0]:
%sql
select * from retail_databricks_catalog.gold_schema.dimproducts;

In [0]:
%sql
select * from retail_databricks_catalog.gold_schema.dimcustomers

In [0]:
df_dimcus = spark.sql("select DimCustomerKey , customer_id as dim_customer_id from retail_databricks_catalog.gold_schema.dimcustomers")
df_dimcus.display()
df_dimpro = spark.sql("select product_id as DimProductKey , product_id as dim_product_id from retail_databricks_catalog.gold_schema.dimproducts")
df_dimpro.display()

In [0]:
df_fact = df.join(df_dimcus, df.customer_id == df_dimcus.dim_customer_id, 'left').join(df_dimpro, df.product_id == df_dimpro.dim_product_id, 'left')


df_fact_new = df_fact.drop("dim_customer_id" , "dim_product_id" , "customer_id" , "product_id")
df_fact_new.display()


### Upsert on Fact Table

In [0]:
from delta.tables import DeltaTable


In [0]:
if spark.catalog.tableExists("retail_databricks_catalog.gold_schema.FactOrders"):
  
  dlt_obj = DeltaTable.forName(spark , "retail_databricks_catalog.gold_schema.FactOrders")
  
  dlt_obj.alias("trg").merge(
    df_fact_new.alias("src"),
    "trg.order_id = src.order_id AND trg.DimCustomerKey = src.DimCustomerKey AND trg.DimProductKey = src.DimProductKey")\
        .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
        .execute()
  

else:
    df_fact_new.write.format("delta")\
        .option("path" , "abfss://gold@stretaildataproj.dfs.core.windows.net/FactOrders/")\
        .saveAsTable("retail_databricks_catalog.gold_schema.FactOrders")
    

In [0]:
%sql
select * from retail_databricks_catalog.gold_schema.factorders